# Linear Algebra with Cython: Interface to BLAS/LAPACK

Martin Lysy *mlysy@uwaterloo.ca*

September 22, 2019

## Preliminaries

1.  Install **scipy** and **cython**.  I did this with **conda**:
    ```
    conda create --name cylinenv python=3.7.4 scipy cython
    ```
2.  Install a C compiler.  It seems this was already done on my computer.

3.  Reading material:

    - [cython](https://cython.readthedocs.io/en/latest/index.html) documentation.  Contains installation notes, various forms of compiling, static typing, etc. 
    - [BLAS](http://www.netlib.org/blas/) and [LAPACK](http://www.netlib.org/lapack/lug/node1.html) documentation.  The "Structure of LAPACK" in the latter explains the naming conventions used in both (I think we want the "d" = `double` routines for interfacing with Python).
    - BLAS/LAPACK [function documentation](http://www.netlib.org/lapack/explore-html/modules.html).  This specifies the exact input/outputs for the functions overviewed above.
    - [cython for NumPy users](https://cython.readthedocs.io/en/latest/src/userguide/numpy_tutorial.html): tips on interfacing with `ndarray`s.
    - SciPy [low-level routines](https://docs.scipy.org/doc/scipy/reference/linalg.html#low-level-routines): the functions in `scipy.linalg.cython_{blas/lapack}` give direct access to the corresponding Fortran functions for cythonizing.
    - NumPy [internals](https://docs.scipy.org/doc/numpy/reference/internals.html): discusses what to do about "conventional python" vs "Fortran" ordering of arrays.
    - Discussion of Cython [memoryviews](http://docs.cython.org/en/latest/src/userguide/memoryviews.html), which shows how to interface with NumPy arrays without unnecessary copying.
    - Explanation of the [leading dimension](https://scc.ustc.edu.cn/zlsc/tc4600/intel/2017.0.098/mkl/common/mklman_c/GUID-E0EC22B2-FDDF-4340-8B6E-C4E0747626B2.htm) argument of BLAS/LAPACK routines for accessing submatrices of larger matrices.

## Hello World Example

Taken from [here](https://cython.readthedocs.io/en/latest/src/tutorial/cython_tutorial.html).

In [1]:
import pyximport
pyximport.install()
import helloworld

ModuleNotFoundError: No module named 'helloworld'

If we want to compile the code for reuse, need `setup.py`:

In [4]:
!python setup.py build_ext --inplace

Compiling kalman_ode_higher.pyx because it changed.
Compiling kalman_ode_higher.py because it changed.
[1/2] Cythonizing kalman_ode_higher.py
[2/2] Cythonizing kalman_ode_higher.pyx
running build_ext
building 'kalman_ode_higher' extension
creating build
creating build\temp.win-amd64-3.7
creating build\temp.win-amd64-3.7\Release
C:\Program Files (x86)\Microsoft Visual Studio\2019\BuildTools\VC\Tools\MSVC\14.24.28314\bin\HostX86\x64\cl.exe /c /nologo /Ox /W3 /GL /DNDEBUG /MT -IC:\Users\mohan\Anaconda3\lib\site-packages\numpy\core\include -IC:\Users\mohan\Anaconda3\include -IC:\Users\mohan\Anaconda3\include "-IC:\Program Files (x86)\Microsoft Visual Studio\2019\BuildTools\VC\Tools\MSVC\14.24.28314\include" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.18362.0\ucrt" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.18362.0\shared" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.18362.0\um" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.18362.0\winrt" "-IC:\Prog

C:\Users\mohan\Anaconda3\lib\site-packages\Cython\Compiler\Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: C:\Users\mohan\Documents\kalmantv\cythonize\kalman_ode_higher.py
  tree = Parsing.p_module(s, pxd, full_module_name)
C:\Users\mohan\Anaconda3\lib\site-packages\Cython\Compiler\Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: C:\Users\mohan\Documents\kalmantv\cythonize\kalman_ode_higher.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [3]:
import helloworld

ModuleNotFoundError: No module named 'helloworld'

Something a bit strange with printing, i.e., won't do it twice in a row...

Let's see if it's the same with something that returns a number:

In [10]:
import numpy as np # for random draws
import fibonacci # cython version
# python version
def fib2(n):
    """Print the Fibonacci series up to n."""
    a, b = 0, 1
    while b < n:
        print(b, end=' ')
        a, b = b, a + b
    return b

n = np.random.randint(50)
fibonacci.fib(n) - fib2(n)

1 1 2 3 5 8 13 21 34 

0

This works as expected.  No `.so` file generated from "inline" `import` call (instead of via `setup.py`).

## Linear Algebra

OK let's start by calling one of the **BLAS**/**LAPACK** functions directly from **scipy**.  Let's try solving a linear system $X = V^{-1}B$ where $V$ is a variance matrix (symmetric positive definite).

In [6]:
import numpy as np
import scipy as sp
import scipy.linalg

def solveV_py(V, B):
    """Solve linear system with variance matrix."""
    U , _ = sp.linalg.lapack.dpotrf(V, False, False) # cholesky decomposition
    X , info = sp.linalg.lapack.dpotrs(U, B) # solve system
    return X

# verify solution
n = 5
p = 2
V = np.random.rand(n,n)
V = np.matmul(V, V.T)
B = np.random.rand(n, p) # RHS

sp.linalg.solve(V, B) - solveV_py(V, B)

array([[ 3.83693077e-13, -5.86197757e-13],
       [-5.57776048e-13,  8.66862138e-13],
       [-4.44089210e-13,  6.89226454e-13],
       [ 6.53699317e-13, -1.01607611e-12],
       [-1.38555833e-13,  2.14939178e-13]])

OK let's build this with Cython.  Here's a couple of things to worry about:

1.  BLAS/LAPACK often does operations in-place, i.e., will probably need to do some copying. 

2.  The BLAS/LAPACK functions assume Fortran-style matrices (column-major), whereas NumPy is row-major by default.

Here's the example, using yet another way of compiling on-the-fly within jupyter notebooks.

In [7]:
%load_ext Cython

In [8]:
%%cython

from scipy.linalg.cython_lapack cimport dpotrf, dpotrs

cpdef solveV(double[::1,:] V, double[::1,:] B, double[::1,:] X, double[::1,:] U):
    """Solve a linear system with a variance matrix.
    Note the matrix U, which is provided for intermediate storage (otherwise things get overwritten).
    """
    # get dimensions
    cdef int n=V.shape[0], nrhs=B.shape[1], lda=n, ldb=n, info
    cdef char* uplo='U'
    # cholesky factor
    U[:] = V # operates in-place, so this prevents V from being overwritten
    dpotrf(&uplo[0],&n,&U[0,0],&lda,&info)
    # solve system with cholesky factor
    X[:] = B # again prevents overwriting
    dpotrs(&uplo[0],&n,&nrhs,&U[0,0],&lda,&X[0,0],&ldb,&info)

cpdef solveV_multi(double[::1,:,:] V, double[::1,:,:] B, double[::1,:,:] X, double[::1,:] U):
    """Multiple calls to solveV"""
    cdef int nreps = V.shape[2], ii
    for ii in range(nreps):
        solveV(V[:,:,ii], B[:,:,ii], X[:,:,ii], U)


In [9]:
# test code
# make sure everything is column-major order!
n = np.random.randint(2,10)
p = np.random.randint(2,10)
V = np.zeros((n,n), order='F')
V[:] = np.random.rand(n,n)
V[:] = np.matmul(V, V.T)
B = np.array(np.random.rand(n,p),order='F')
print(np.isfortran(V))
print(np.isfortran(B))

True
True


In [14]:
# test code
# make sure everything is column-major order!
n = np.random.randint(2,10)
p = np.random.randint(2,10)
V = np.zeros((n,n), order='F')
V[:] = np.random.rand(n,n)
V[:] = np.matmul(V, V.T)
B = np.array(np.random.rand(n,p),order='F')
print(np.isfortran(V))
print(np.isfortran(B))

# do calculations in python first to make sure there's no overwriting
Up = sp.linalg.cholesky(V)
Xp = sp.linalg.solve(V, B)

# first pure cholesky
U = np.zeros((n,n), order='F')
print(np.isfortran(U))
chol(V, U)
print(Up - np.triu(U))

# now solver
X = np.zeros((n,p), order='F')
solveV(V, B, X, U)
print(Xp - X)

True
True
True
[[0. 0.]
 [0. 0.]]
[[ 2.22044605e-16  2.22044605e-16  4.44089210e-16]
 [ 1.11022302e-16 -3.46944695e-18 -1.38777878e-17]]


### Speed Test

Now we have `solveV` written entirely in C, and `solveV_py` using the same C-level functions but with only a Python interface.  Therefore, if we want to loop the calculation over several matrices, the latter will have some overhead.  Let's see how much this overhead is.

In [70]:
# preliminary functions

# lapack/python solver
def solveV_multi_py(V, B, X):
    nreps = V.shape[2]
    for ii in range(nreps):
        X[:,:,ii] = solveV_py(V[:,:,ii], B[:,:,ii])
        
# relative error
def rel_err(X1, X2):
    return np.max(np.abs((X1.ravel() - X2.ravel())/X1.ravel()))

# timing function
from contextlib import contextmanager
from time import time

@contextmanager
def timing(description: str) -> None:
    start = time()
    yield
    ellapsed_time = time() - start

    print(f"{description}: {ellapsed_time}")


In [86]:
# generate random data
#n = np.random.randint(2,10) # size of matrix
#p = np.random.randint(2,10) # size of solution
#nreps = np.random.randint(5,10) # number of replications
n = 20
p = 1
nreps = 1000

V = np.zeros((n,n,nreps), order='F')
for ii in range(nreps):
    vv = np.random.rand(n,n)
    V[:,:,ii] = np.matmul(vv, vv.T)
B = np.array(np.random.rand(n,p,nreps),order='F')

# first check in pure Python to make sure everything works correctly
Xp1 = np.zeros((n,p,nreps), order='F')
with timing("Pure SciPy ()"):
    for ii in range(nreps):
        C, low = sp.linalg.cho_factor(V[:,:,ii])
        Xp1[:,:,ii] = sp.linalg.cho_solve((C, low), B[:,:,ii])

# now check lapack/python
Xp2 = np.zeros((n,p,nreps), order='F')
with timing("LAPACK/SciPy"):
    solveV_multi_py(V, B, Xp2)

# finally, lapack/cython
X = np.zeros((n,p,nreps), order='F')
U = np.zeros((n,n), order='F') # temporary storage

with timing("Pure LAPACK"):
    solveV_multi(V, B, X, U)

print(f"relative error: {rel_err(Xp1, Xp2)}")
print(f"relative error: {rel_err(Xp1, X)}")

Pure SciPy: 0.040476322174072266
LAPACK/SciPy: 0.005300998687744141
Pure LAPACK: 0.0017158985137939453
relative error: 0.0
relative error: 0.0


In [57]:
X = np.zeros((n,p,nreps), order='F')
for ii in range(nreps):
    Xp[:,:,ii] = sp.linalg.solve(V[:,:,ii], B[:,:,ii])
    solveV(V[:,:,ii], B[:,:,ii], X[:,:,ii], U)

np.max(np.abs(Xp.ravel() - X.ravel())/np.abs(Xp.ravel()))

## Scratch

Before jumping in with `solveV` I tried a simpler LAPACK function, namely solving a tridiagonal system.  It's simpler because it doesn't require matrix inputs, so we don't have to worry about order.

In [ ]:
%%cython

from scipy.linalg.cython_lapack cimport dgtsv

cpdef tridiag(double[::1] a, double[::1] b, 
              double[::1] c, double[::1] x):
    cdef int n=b.shape[0], nrhs=1, info
    # Solution is written over the values in x. 
    dgtsv(&n, &nrhs, &a[0], &b[0], &c[0], &x[0], &n, &info)

In [ ]:
# test code
def tridiag_py(a, b, c, x):
    """ Solve the system A y = x for y
    where A is the square matrix with subdiagonal 'a', 
    diagonal 'b', and superdiagonal 'c'. """ 
    A = np.zeros((b.shape[0], b.shape[0])) 
    np.fill_diagonal(A[1:], a) 
    np.fill_diagonal(A, b) 
    np.fill_diagonal(A[:,1:], c)
    return np.linalg.solve(A, x)

n = 10
a = np.random.rand(n-1)
b = np.random.rand(n)
c = np.random.rand(n-1)
x = np.random.rand(n)

y1 = tridiag_py(a, b, c, x)
tridiag(a, b, c, x)
x - y1

This is just some tests of NumPy array ordering.

In [42]:
X = np.random.rand(2,2)
Y = np.copy(X, order='F')
print(X)
print(Y)
print(X.flags)
print(Y.flags)

[[0.39949194 0.36282424]
 [0.75187443 0.40503242]]
[[0.39949194 0.36282424]
 [0.75187443 0.40503242]]
  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False
  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False


Here I just want to confirm that `solveV` accepts submatrices.

In [47]:
# test code
# make sure everything is column-major order!
n = np.random.randint(2,10) # size of matrix
p = np.random.randint(2,10) # size of solution
nreps = np.random.randint(5,10) # number of replications
V = np.zeros((n,n,nreps), order='F')
for ii in range(nreps):
    vv = np.random.rand(n,n)
    V[:,:,ii] = np.matmul(vv, vv.T)
B = np.array(np.random.rand(n,p,nreps),order='F')
U = np.zeros((n,n), order='F')
print(np.isfortran(V))
print(np.isfortran(B))

Xp = np.zeros((n,p,nreps), order='F')
X = np.zeros((n,p,nreps), order='F')
for ii in range(nreps):
    Xp[:,:,ii] = sp.linalg.solve(V[:,:,ii], B[:,:,ii])
    solveV(V[:,:,ii], B[:,:,ii], X[:,:,ii], U)

np.max(np.abs(Xp.ravel() - X.ravel())/np.abs(Xp.ravel()))

True
True


7.68799009161521e-14

In [23]:


# do calculations in python first to make sure there's no overwriting
Up = sp.linalg.cholesky(V)
Xp = sp.linalg.solve(V, B)

# first pure cholesky
U = np.zeros((n,n), order='F')
print(np.isfortran(U))
chol(V, U)
print(Up - np.triu(U))

# now solver
X = np.zeros((n,p), order='F')
solveV(V, B, X, U)
print(Xp - X)

ValueError: Input array needs to be 2 dimensional but received a 3d-array.